In [1]:
import gym
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam
import random

Using TensorFlow backend.


In [2]:
env = gym.make('CartPole-v1')
env.reset()
goal_steps = 500
score_requirement = 60
intial_games = 10000

In [3]:
def playgame():
    for step_index in range(goal_steps):
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
        if done:
            break
    env.reset()

In [4]:
playgame()

Step 0:
action: 0
observation: [ 0.02672348 -0.20565988  0.04800755  0.25844915]
reward: 1.0
done: False
info: {}
Step 1:
action: 0
observation: [ 0.02261028 -0.40143316  0.05317653  0.56587953]
reward: 1.0
done: False
info: {}
Step 2:
action: 1
observation: [ 0.01458162 -0.20709597  0.06449412  0.29041233]
reward: 1.0
done: False
info: {}
Step 3:
action: 0
observation: [ 0.0104397  -0.40307543  0.07030237  0.60271848]
reward: 1.0
done: False
info: {}
Step 4:
action: 0
observation: [ 0.00237819 -0.59910663  0.08235674  0.9166908 ]
reward: 1.0
done: False
info: {}
Step 5:
action: 0
observation: [-0.00960394 -0.7952398   0.10069055  1.23407892]
reward: 1.0
done: False
info: {}
Step 6:
action: 0
observation: [-0.02550874 -0.9915016   0.12537213  1.55653354]
reward: 1.0
done: False
info: {}
Step 7:
action: 0
observation: [-0.04533877 -1.18788255  0.1565028   1.88555585]
reward: 1.0
done: False
info: {}
Step 8:
action: 0
observation: [-0.06909642 -1.38432268  0.19421392  2.22243906]
reward:

In [5]:
def modeldp():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])                
            previous_observation = observation
            score += reward
            if done:
                break            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                training_data.append([data[0], output])
        
        env.reset()

    print(sum(accepted_scores)/len(accepted_scores))   
    return training_data

In [6]:
training_data = modeldp()

70.37142857142857


In [7]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(1024, input_dim=input_size, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model

In [8]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    model.fit(X, y, epochs=20)
    return model

In [9]:
trained_model = train_model(training_data)







Epoch 1/20
9712/9712 [==============================] - 2s 203us/step - loss: 0.2463
Epoch 2/20
9712/9712 [==============================] - 2s 174us/step - loss: 0.2355
Epoch 3/20
9712/9712 [==============================] - 2s 178us/step - loss: 0.2351
Epoch 4/20
9712/9712 [==============================] - 2s 177us/step - loss: 0.2332
Epoch 5/20
9712/9712 [==============================] - 2s 168us/step - loss: 0.2331
Epoch 6/20
9712/9712 [==============================] - 2s 179us/step - loss: 0.2325
Epoch 7/20
9712/9712 [==============================] - 2s 190us/step - loss: 0.2319
Epoch 8/20
9712/9712 [==============================] - 2s 172us/step - loss: 0.2319
Epoch 9/20
9712/9712 [==============================] - 2s 180us/step - loss: 0.2313
Epoch 10/20
9712/9712 [==============================] - 2s 170us/step - loss: 0.2312
Epoch 11/20
9712/9712 [==============================] - 2s 170us/step - loss: 0.2311
Epoch 12/20
9712/9712 [==============================] - 

In [10]:
scores =  []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1,len(prev_obs)))[0])
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=reward
        if done:
            break
    env.reset()
    scores.append(score)
print(scores)
print('Average Score:',sum(scores)/len(scores))

[152.0, 196.0, 93.0, 95.0, 78.0, 266.0, 84.0, 119.0, 74.0, 132.0, 91.0, 147.0, 90.0, 92.0, 115.0, 206.0, 198.0, 110.0, 116.0, 187.0, 187.0, 86.0, 122.0, 122.0, 204.0, 500.0, 163.0, 90.0, 234.0, 80.0, 139.0, 138.0, 107.0, 128.0, 126.0, 84.0, 177.0, 321.0, 237.0, 209.0, 252.0, 93.0, 186.0, 125.0, 84.0, 99.0, 84.0, 98.0, 228.0, 256.0, 210.0, 148.0, 162.0, 88.0, 160.0, 86.0, 206.0, 92.0, 88.0, 248.0, 122.0, 77.0, 146.0, 182.0, 141.0, 125.0, 92.0, 495.0, 192.0, 202.0, 214.0, 92.0, 109.0, 132.0, 116.0, 146.0, 500.0, 222.0, 74.0, 104.0, 68.0, 108.0, 152.0, 104.0, 93.0, 92.0, 98.0, 88.0, 308.0, 105.0, 271.0, 196.0, 88.0, 130.0, 91.0, 157.0, 260.0, 213.0, 120.0, 140.0]
Average Score: 154.53
